In [1]:
import os
import glob
import cv2
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
from scipy.stats import linregress

In [ ]:
# Phantom Flow

### Importing Images

cropped = 'croppedimg_path'

# Get a list of all files in the folder matching the pattern
filePattern = os.path.join(cropped, "*.bmp")

theFiles = glob.glob(filePattern)
theFiles = sorted(glob.glob(filePattern))

exposure1ms = []
exposure10ms = []

for fullFileName in theFiles:
    baseName = os.path.basename(fullFileName)
    if "trial2" in baseName:
        exposure10ms.append(fullFileName)
    else:
        exposure1ms.append(fullFileName)

for fullFileName in exposure1ms:
    print(f"Now reading (1ms Exposure): {fullFileName}")
    imageArray = cv2.imread(fullFileName)

for fullFileName in exposure10ms:
    print(f"Now reading (10m Exposure): {fullFileName}")
    imageArray = cv2.imread(fullFileName)

### LSC calculation function

def compute_lsc(image, window_size):
    image = image.astype(np.float32)
    
    # Local mean and standard deviation
    mean = uniform_filter(image, size=window_size)
    mean_sq = uniform_filter(image**2, size=window_size)
    std = np.sqrt(mean_sq - mean**2)
    
    # Calculate speckle contrast map
    lsc = np.where(mean != 0, std / mean, 0)
    
    return lsc

### Defining Region of Interest (ROI) using Thresholding

def extract_tube_roi(gray_img):
    # Step 1: Apply Gaussian blur to smooth the image
    blurred = cv2.GaussianBlur(gray_img, (9, 9), 0)

    # Step 2: Use adaptive thresholding to capture varying brightness along the tube
    adaptive_thresh = cv2.adaptiveThreshold(
        blurred, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        11, 2  # Block size and constant
    )

    # Step 3: Invert the image if necessary (tube is light on dark background)
    inverted = cv2.bitwise_not(adaptive_thresh)

    # Step 4: Morphological filtering to isolate the narrow tube
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 3))  # Long narrow kernel
    closed = cv2.morphologyEx(inverted, cv2.MORPH_CLOSE, kernel)

    # Step 5: Optional - remove small noise blobs
    cleaned = cv2.morphologyEx(closed, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # Step 6: Apply the mask to the original grayscale image
    tube_region = cv2.bitwise_and(gray_img, gray_img, mask=cleaned)

    return tube_region, cleaned

# List to store (filename, K value) results
exposure1ms_K = []
window_size = 7

# Loop over breath held images
for fullFileName in exposure1ms:
    print(f"Processing: {fullFileName}")
    
    # Read as grayscale
    img = cv2.imread(fullFileName, cv2.IMREAD_GRAYSCALE)

    # Extract finger region and its mask
    tube_roi, mask = extract_tube_roi(img)

    # Compute LSC map on finger region
    lsc_map = compute_lsc(img, window_size)

    # Compute mean K only where mask > 0 (i.e., tube region)
    mean_K = np.mean(lsc_map[mask > 0])
    
    print(f"LSC K value (tube only): {mean_K:.4f}")
    exposure1ms_K.append(mean_K)

#### List to store (filename, K value) results
exposure10ms_K = []
window_size = 7

# Loop over breath held images
for fullFileName in exposure10ms:
    print(f"Processing: {fullFileName}")
    
    # Read as grayscale
    img = cv2.imread(fullFileName, cv2.IMREAD_GRAYSCALE)

    # Extract finger region and its mask
    tube_roi, mask = extract_tube_roi(img)

    # Compute LSC map on finger region
    lsc_map = compute_lsc(img, window_size)

    # Compute mean K only where mask > 0 (i.e., tube region)
    mean_K = np.mean(lsc_map[mask > 0])
    
    print(f"LSC K value (tube only): {mean_K:.4f}")
    exposure10ms_K.append(mean_K)

### Plotting

velocities = [0,2,4,6,8,10]

# Plot 1: Exposure 1 ms
plt.figure(figsize=(8, 5))
plt.plot(velocities, exposure1ms_K, 'o-', color='blue', label='Exposure 1 ms', linewidth=2)
plt.xlabel('Velocity (mm/s)', fontsize=12)
plt.ylabel('LSCI Parameter K Value', fontsize=12)
plt.title('K Values at Different Velocities (Exposure: 1 ms)', fontsize=14)
plt.grid(True)
plt.figtext(0.5, -0.1,
            'Figure 1: K values measured at 1 ms exposure across varying flow velocities.',
            wrap=True, horizontalalignment='center', fontsize=10)
plt.tight_layout()
plt.show()

# Plot 2: Exposure 10 ms
plt.figure(figsize=(8, 5))
plt.plot(velocities, exposure10ms_K, 'o-', color='blue', label='Exposure 10 ms', linewidth=2)
plt.xlabel('Velocity (mm/s)', fontsize=12)
plt.ylabel('LSCI Parameter K Value', fontsize=12)
plt.title('K Values at Different Velocities (Exposure: 10 ms)', fontsize=14)
plt.grid(True)
plt.legend()
plt.figtext(0.5, -0.1,
            'Figure 2: K values measured at 10 ms exposure across varying flow velocities.',
            wrap=True, horizontalalignment='center', fontsize=10)
plt.tight_layout()
plt.show()

### Pearson Correlation coefficient

r_1ms, p_1ms = pearsonr(velocities, exposure1ms_K)
print(f"Pearson correlation (1 ms): r = {r_1ms:.3f}, p-value = {p_1ms:.3e}")

# Correlation for 10 ms exposure
r_10ms, p_10ms = pearsonr(velocities, exposure10ms_K)
print(f"Pearson correlation (10 ms): r = {r_10ms:.3f}, p-value = {p_10ms:.3e}")

### Linear Regression and Statistical Analysis

slope1, intercept1, r1, p1, stderr1 = linregress(velocities, exposure1ms_K)

# Linear regression for 10 ms
slope10, intercept10, r10, p10, stderr10 = linregress(velocities, exposure10ms_K)

r2_1 = r1**2
r2_10 = r10**2
print(r2_1, r2_10)
print(slope1, slope10)

# t-statistic for difference between slopes
t_slope = abs(slope1 - slope10) / np.sqrt(stderr1**2 + stderr10**2)

# Degrees of freedom: n1 + n2 - 4 (each linear regression uses 2 df)
from scipy.stats import t
df = len(velocities)*2 - 4
p_diff = 2 * (1 - t.cdf(t_slope, df))

p_diff